In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])

for name in os.listdir('./Model/'):
    #print(name.split('.')[0][-1].isdigit())
    #print(name)
    if name.split('.')[0][-1].isdigit() or name.startswith('Linear')or name.startswith('Forest'):
        full_name=name
        model=torch.load(f'./Model/{name}')
        name=name.split('_')[1]
        name=name.replace('.pth','')
        #print('FIRST',name)
        #print(name[:-1])
        if name[-1].isdigit():
            name=name[:-1]
        #print('Second',name)

        dataset = pd.read_csv(f'../data/{name}/{name}.csv')
        continuous_wachter = dataset.drop(columns=['label']).columns
        scaler='Identity'
        if name == 'economic' or name == 'nutrition':
            scaler='MinMax'

        dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        dataset=dataset._df_test
        data_x, data_y=dataset.drop(columns=['label']), dataset['label']
        data_pred= model.predict(data_x)
        model_name.append(full_name)
        print(data_pred.shape)
        print(data_y.shape)
        if len(data_pred.shape)>1:
            data_pred= np.argmax(data_pred, axis=1)

        accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
        precision.append(sklearn.metrics.precision_score(data_y,data_pred))
        recall.append(sklearn.metrics.recall_score(data_y,data_pred))
        auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)


...

(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.816,0.839537,0.79235,0.816601
2,Forest_economic.pth,0.8076,0.757702,0.827273,0.809708
3,Forest_sanity-3-lin.pth,0.8084,0.805924,0.818038,0.808291
4,Forest_sanity-3-non-add.pth,0.8608,0.907407,0.480392,0.732251
5,Forest_sanity-3-non-lin.pth,0.9692,0.977954,0.955211,0.96827
6,Linear_credit.pth,0.7572,0.770478,0.727639,0.756989
7,Linear_economic.pth,0.8272,0.793852,0.81146,0.825335
8,Linear_nutrition.pth,0.8832,0.856154,0.913793,0.883964
9,Linear_sanity-3-lin.pth,0.9968,0.996956,0.996956,0.996792


In [20]:
df_new=pd.DataFrame([])
model=[]
dataset=[]
acc=[]
for index, row in df.iterrows():
    model.append(row['model_name'].split('_')[0])
    d=row['model_name'].split('_')[1].replace('.pth','')
    if 'MLP' in row['model_name'].split('_')[0]:
        d=row['model_name'].split('_')[1].replace('.pth','')[:-1]
    dataset.append(d)
    acc.append(row['auc'])

df_new['model']=model
df_new['dataset']=dataset
df_new['auc']=acc
df_new.to_csv('Classifier_results.csv')


In [18]:
st=''
n=[]
for dataset in np.unique(df_new['dataset']):
    st += f'& {dataset}'
    n.append(dataset)
st+='\\\\'

for m in np.unique(df_new['model']):
    st+=f'{m}'
    m_m=df_new[df_new['model']==m]
    for i in n:
        try: 
            a=round(m_m[m_m['dataset']==i]['auc'].values[0],2)
            st+=f'& {a}'
        except:
            st+=f'& '

    st+='\\\\'

print(st)



& credit& economic& nutrition& sanity-3-lin& sanity-3-non-add& sanity-3-non-lin\\Forest& 0.82& 0.81& & 0.81& 0.73& 0.97\\Linear& 0.76& 0.83& 0.88& 1.0& 1.0& 1.0\\MLP& 0.69& 0.82& 0.9& 1.0& 0.99& 1.0\\
